In [35]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from langgraph.checkpoint.memory import InMemorySaver
from dotenv import load_dotenv
import os 
load_dotenv()

True

In [36]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite-preview-06-17",
    api_key=os.environ["GEMINI_API_KEY"]
)

In [37]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explaination: str

In [38]:
def generate_joke(state):
    prompt = f"Generate a one line joke on this given topic: {state["topic"]}"
    response = model.invoke(prompt).content
    return {"joke" : response}

In [ ]:
def generate_explaination(state):
    prompt = f"Generate short explaination of given joke:\n {state["joke"]}"
    response = model.invoke(prompt).content
    return {"explaination" : response}

In [44]:
graph = StateGraph(JokeState)
graph.add_node("generate_joke", generate_joke)
graph.add_node("generate_explaination", generate_explaination)
graph.add_edge(START, "generate_joke")
graph.add_edge("generate_joke", "generate_explaination")
graph.add_edge("generate_explaination", END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [48]:
# config1 = {"configurable": {"thread_id": "1"}}
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic': "burger"}, config=config2)

{'topic': 'burger',
 'joke': 'Why did the hamburger break up with the hot dog? Because he felt like he was getting bun-dled.',
 'explaination': 'This joke is a pun, playing on two different meanings of the word "bundled."\n\nHere\'s the breakdown:\n\n*   **Literal meaning of "bundled":** In the context of food, a "bundle" could refer to a group of things wrapped together.\n*   **Figurative meaning of "bundled":** This is where the humor comes in. "Bundled" also sounds like "bun-dled."\n\n**The Joke Explained:**\n\n1.  **The Setup:** "Why did the hamburger break up with the hot dog?" This sets up a scenario of a relationship ending between two common fast-food items.\n\n2.  **The Punchline:** "Because he felt like he was getting bun-dled."\n\n3.  **The Pun:** The hamburger is breaking up with the hot dog because he feels like he\'s being "bun-dled." This means two things simultaneously:\n    *   **Literally:** The hamburger is being put into a bun, and a hot dog is also typically served

In [50]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'burger', 'joke': 'Why did the hamburger break up with the hot dog? Because he felt like he was getting bun-dled.', 'explaination': 'This joke is a pun, playing on two different meanings of the word "bundled."\n\nHere\'s the breakdown:\n\n*   **Literal meaning of "bundled":** In the context of food, a "bundle" could refer to a group of things wrapped together.\n*   **Figurative meaning of "bundled":** This is where the humor comes in. "Bundled" also sounds like "bun-dled."\n\n**The Joke Explained:**\n\n1.  **The Setup:** "Why did the hamburger break up with the hot dog?" This sets up a scenario of a relationship ending between two common fast-food items.\n\n2.  **The Punchline:** "Because he felt like he was getting bun-dled."\n\n3.  **The Pun:** The hamburger is breaking up with the hot dog because he feels like he\'s being "bun-dled." This means two things simultaneously:\n    *   **Literally:** The hamburger is being put into a bun, and a hot dog is al

In [51]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'burger', 'joke': 'Why did the hamburger break up with the hot dog? Because he felt like he was getting bun-dled.', 'explaination': 'This joke is a pun, playing on two different meanings of the word "bundled."\n\nHere\'s the breakdown:\n\n*   **Literal meaning of "bundled":** In the context of food, a "bundle" could refer to a group of things wrapped together.\n*   **Figurative meaning of "bundled":** This is where the humor comes in. "Bundled" also sounds like "bun-dled."\n\n**The Joke Explained:**\n\n1.  **The Setup:** "Why did the hamburger break up with the hot dog?" This sets up a scenario of a relationship ending between two common fast-food items.\n\n2.  **The Punchline:** "Because he felt like he was getting bun-dled."\n\n3.  **The Pun:** The hamburger is breaking up with the hot dog because he feels like he\'s being "bun-dled." This means two things simultaneously:\n    *   **Literally:** The hamburger is being put into a bun, and a hot dog is a

### Time travel

In [ ]:

workflow.get_state({"configurable": {"thread_id": "2", "checkpoint_id": "1f06dd79-841e-65be-8000-5e6ee1e9c924"}})

StateSnapshot(values={'topic': 'burger'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_id': '1f06dd79-841e-65be-8000-5e6ee1e9c924'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-07-31T06:28:38.833692+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f06dd79-841c-6952-bfff-98da49271480'}}, tasks=(PregelTask(id='24206c07-651a-b00e-7927-380de7b91631', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the hamburger break up with the hot dog? Because he felt like he was getting bun-dled.'}),), interrupts=())

In [55]:
workflow.invoke(None, {"configurable": {"thread_id": "2", "checkpoint_id": "1f06dd79-841e-65be-8000-5e6ee1e9c924"}})

{'topic': 'burger',
 'joke': 'Why did the burger break up with the bun? It felt smothered!',
 'explaination': 'This joke plays on the word "smothered" and its double meaning:\n\n*   **Literal Meaning:** In the context of food, "smothered" often means covered in a sauce or topping, like "smothered fries" or "smothered chicken." A burger and bun are often put together, and the burger can be seen as being "smothered" by the bun, especially if it\'s a tight fit or if there are many toppings inside.\n\n*   **Figurative Meaning:** In relationships, "smothered" means feeling overwhelmed, suffocated, or controlled by someone\'s affection or attention. It implies a lack of space and independence.\n\n**The Joke\'s Humor:**\n\nThe humor comes from applying the **figurative, relationship meaning** of "smothered" to the **literal, food context** of a burger and bun. The burger is personified as having feelings, and it breaks up with the bun because it feels overwhelmed by being enclosed within it, 

In [56]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'burger', 'joke': 'Why did the burger break up with the bun? It felt smothered!', 'explaination': 'This joke plays on the word "smothered" and its double meaning:\n\n*   **Literal Meaning:** In the context of food, "smothered" often means covered in a sauce or topping, like "smothered fries" or "smothered chicken." A burger and bun are often put together, and the burger can be seen as being "smothered" by the bun, especially if it\'s a tight fit or if there are many toppings inside.\n\n*   **Figurative Meaning:** In relationships, "smothered" means feeling overwhelmed, suffocated, or controlled by someone\'s affection or attention. It implies a lack of space and independence.\n\n**The Joke\'s Humor:**\n\nThe humor comes from applying the **figurative, relationship meaning** of "smothered" to the **literal, food context** of a burger and bun. The burger is personified as having feelings, and it breaks up with the bun because it feels overwhelmed by being 

### Updating state


In [58]:
workflow.update_state({"configurable": {"thread_id": "2", "checkpoint_id": "1f06dd79-841e-65be-8000-5e6ee1e9c924", "checkpoint_ns": ""}}, {"topic": "samosa"})

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f06ddba-eccc-615f-8001-2658dc941d60'}}

In [59]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f06ddba-eccc-615f-8001-2658dc941d60'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-07-31T06:57:54.640510+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f06dd79-841e-65be-8000-5e6ee1e9c924'}}, tasks=(PregelTask(id='eff177a8-c01a-1ccf-8763-1b9d1e697740', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'burger', 'joke': 'Why did the burger break up with the bun? It felt smothered!', 'explaination': 'This joke plays on the word "smothered" and its double meaning:\n\n*   **Literal Meaning:** In the context of food, "smothered" often means covered in a sauce or topping, like "smothered fries" or "smothered chicken." A burger and bun are often put 

In [61]:
workflow.invoke(None, {"configurable": {"thread_id": "2", "checkpoint_id": "1f06ddba-eccc-615f-8001-2658dc941d60"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa break up with the chutney? Because it felt like the relationship was getting too *spicy* and it needed some *space*!',
 'explaination': 'This joke plays on the words "spicy" and "space" to create a humorous, albeit silly, reason for a breakup. Here\'s the breakdown:\n\n*   **Samosa:** A popular Indian snack, typically a fried or baked pastry with a savory filling, often containing potatoes, peas, and spices.\n*   **Chutney:** A condiment, often made from fruits, vegetables, and spices, that is commonly served with samosas. Chutneys can range in flavor from sweet to sour to very spicy.\n\nNow, let\'s look at the punchline:\n\n*   **"Because it felt like the relationship was getting too *spicy*..."**\n    *   **Literal Meaning (Food):** Samosas and chutneys are often served together, and many chutneys are indeed spicy. So, in a food context, a "spicy" relationship between a samosa and chutney is a very natural and expected pairing.\n    * 